In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
data = pd.read_csv('../data/mini.csv', parse_dates=['occurence'])
data.head()

,customer_id,occurence,cost,item_id
0,416705,2017-05-07 21:58:10,299.0,515274
1,13891,2018-02-10 17:35:11,1090.0,828115
2,9081,2017-12-21 17:13:44,499.0,695501
3,470904,2017-10-31 10:39:49,290.0,899821
4,58500,2018-03-09 20:57:29,150.0,518554


In [3]:
print(data['occurence'].min(), data['occurence'].max())

2017-01-01 12:33:40 2018-07-23 10:38:37


In [4]:
sd = dt.datetime(2018,8,1)
data['recency_in_days']= sd - data['occurence']
data['recency_in_days'] = data['recency_in_days'].astype('timedelta64[D]')
data.head()

,customer_id,occurence,cost,item_id,recency_in_days
0,416705,2017-05-07 21:58:10,299.0,515274,450.0
1,13891,2018-02-10 17:35:11,1090.0,828115,171.0
2,9081,2017-12-21 17:13:44,499.0,695501,222.0
3,470904,2017-10-31 10:39:49,290.0,899821,273.0
4,58500,2018-03-09 20:57:29,150.0,518554,144.0


In [5]:
data = data[data['recency_in_days'] < 730]

In [30]:
rfm_table_users = data.groupby(['item_id', 'customer_id']).agg({'recency_in_days': lambda x: x.min(), # Recency
                                            'customer_id': lambda x: len(x), # Frequency
                                            'cost': lambda x: x.sum()}) # Monetary_value
rfm_table_users.rename(columns={'recency_in_days': 'recency',
                         'customer_id': 'frequency',
                         'cost': 'monetary_value'}, inplace=True)

In [31]:
rfm_table_users

recency  frequency  monetary_value
item_id customer_id                                    
000006  429842         436.0          1          1520.0
000030  502            479.0          1          1791.0
000059  53615          527.0          1           990.0
        102820         351.0          1           687.0
000061  35139          255.0          1           995.0
        46756          370.0          1           945.0
        53615          527.0          1           990.0
        207857         535.0          1           990.0
        272143         315.0          1           141.0
        282037         500.0          1           990.0
        298300         502.0          1           995.0
        442104         290.0          1           877.0
        443259         397.0          1           990.0
        527722         178.0          1           990.0
000064  285             44.0          1           895.0
        397            180.0          1           995.0
        4123           115.0          1           895.0
        109038         247.0          1           945.0
        275571         423.0          1           995.0
        276748         505.0          1           995.0
        279642          95.0          1           945.0
        323025         192.0          1           995.0
        393967          29.0          1           995.0
000160  3016           330.0          1          1395.0
        7418           359.0          1          1325.0
        9227           114.0          1           795.0
        24141          375.0          1          1255.0
        31082          570.0          1          1395.0
        81427          226.0          1          1395.0
        111495         355.0          1          1325.0
...                      ...        ...             ...
cert_5  504760         145.0          1          5000.0
        506073         220.0          1          5000.0
        509292         222.0          1        325000.0
        511849         214.0          1         20000.0
        512924         214.0          1         20000.0
        522742          42.0          1         15000.0
        533348         142.0          1        385000.0
        533677         173.0          1         45000.0
        534822         167.0          1          5000.0
        537722         150.0          1         10000.0
        537772         143.0          1          5000.0
        539262         160.0          1        145000.0
        539270         161.0          1        135000.0
        540075         151.0          1         60000.0
        543277          94.0          1          5000.0
        545100         140.0          1          5000.0
        550716         129.0          1          5000.0
        565585          86.0          1          5000.0
        567420          89.0          1         10000.0
        573511          77.0          1         25000.0
        576643          71.0          1         25000.0
        577159          67.0          1         15000.0
        579557          62.0          1       2599000.0
        579888          56.0          1         10000.0
        580072          62.0          1        675000.0
        582337          33.0          1         10000.0
        584016          47.0          1        200000.0
        584603          39.0          1          5000.0
        585224          57.0          1         50000.0
        605576          27.0          1         10000.0

[1917776 rows x 3 columns]

In [11]:
data[data['customer_id'] == 627746]

,customer_id,occurence,cost,item_id,recency_in_days
359171,627746,2018-07-23 10:05:58,20.0,1016155,8.0


In [32]:
quartiles_users = rfm_table_users.quantile(q=[0.2,0.4,0.6,0.8]).to_dict()

In [33]:
def RClass(x,p,q):
    if x <= q[p][0.2]:
        return 5
    elif x <= q[p][0.4]:
        return 4
    elif x <= q[p][0.6]:
        return 3
    elif x <= q[p][0.8]:
        return 2
    else:
        return 1
    
def FMClass(x,p,q):
    if x <= q[p][0.2]:
        return 1
    elif x <= q[p][0.4]:
        return 2
    elif x <= q[p][0.6]:
        return 3
    elif x <= q[p][0.8]:
        return 4
    else:
        return 5

In [34]:
rfm_seg_users = rfm_table_users
rfm_seg_users['R_Quartile'] = rfm_seg_users['recency'].apply(RClass, args=('recency', quartiles_users))
rfm_seg_users['F_Quartile'] = rfm_seg_users['frequency'].apply(FMClass, args=('frequency',quartiles_users))
rfm_seg_users['M_Quartile'] = rfm_seg_users['monetary_value'].apply(FMClass, args=('monetary_value',quartiles_users))

In [35]:
rfm_seg_users['RFMClass'] = rfm_seg_users.R_Quartile.map(str) \
                            + rfm_seg_users.F_Quartile.map(str) \
                            + rfm_seg_users.M_Quartile.map(str)

In [36]:
rfm_seg_users.head()

recency  frequency  monetary_value  R_Quartile  \
item_id customer_id                                                   
000006  429842         436.0          1          1520.0           2   
000030  502            479.0          1          1791.0           1   
000059  53615          527.0          1           990.0           1   
        102820         351.0          1           687.0           2   
000061  35139          255.0          1           995.0           3   

                     F_Quartile  M_Quartile RFMClass  
item_id customer_id                                   
000006  429842                1           3      213  
000030  502                   1           3      113  
000059  53615                 1           3      113  
        102820                1           2      212  
000061  35139                 1           3      313

In [37]:
rfm_seg_users['Rating'] = rfm_seg_users['R_Quartile'] + rfm_seg_users['F_Quartile'] +rfm_seg_users['M_Quartile']

In [38]:
quartiles_users = rfm_table_users.quantile(q=[0.2,0.4,0.6,0.8]).to_dict()

In [39]:
rfm_seg_users['Rating'] = rfm_seg_users['Rating'].apply(FMClass, args=('Rating', quartiles_users))

In [41]:
rfm_seg_users

recency  frequency  monetary_value  R_Quartile  \
item_id customer_id                                                   
000006  429842         436.0          1          1520.0           2   
000030  502            479.0          1          1791.0           1   
000059  53615          527.0          1           990.0           1   
        102820         351.0          1           687.0           2   
000061  35139          255.0          1           995.0           3   
        46756          370.0          1           945.0           2   
        53615          527.0          1           990.0           1   
        207857         535.0          1           990.0           1   
        272143         315.0          1           141.0           2   
        282037         500.0          1           990.0           1   
        298300         502.0          1           995.0           1   
        442104         290.0          1           877.0           3   
        443259         397.0          1           990.0           2   
        527722         178.0          1           990.0           4   
000064  285             44.0          1           895.0           5   
        397            180.0          1           995.0           4   
        4123           115.0          1           895.0           4   
        109038         247.0          1           945.0           3   
        275571         423.0          1           995.0           2   
        276748         505.0          1           995.0           1   
        279642          95.0          1           945.0           5   
        323025         192.0          1           995.0           4   
        393967          29.0          1           995.0           5   
000160  3016           330.0          1          1395.0           2   
        7418           359.0          1          1325.0           2   
        9227           114.0          1           795.0           4   
        24141          375.0          1          1255.0           2   
        31082          570.0          1          1395.0           1   
        81427          226.0          1          1395.0           3   
        111495         355.0          1          1325.0           2   
...                      ...        ...             ...         ...   
cert_5  504760         145.0          1          5000.0           4   
        506073         220.0          1          5000.0           3   
        509292         222.0          1        325000.0           3   
        511849         214.0          1         20000.0           3   
        512924         214.0          1         20000.0           3   
        522742          42.0          1         15000.0           5   
        533348         142.0          1        385000.0           4   
        533677         173.0          1         45000.0           4   
        534822         167.0          1          5000.0           4   
        537722         150.0          1         10000.0           4   
        537772         143.0          1          5000.0           4   
        539262         160.0          1        145000.0           4   
        539270         161.0          1        135000.0           4   
        540075         151.0          1         60000.0           4   
        543277          94.0          1          5000.0           5   
        545100         140.0          1          5000.0           4   
        550716         129.0          1          5000.0           4   
        565585          86.0          1          5000.0           5   
        567420          89.0          1         10000.0           5   
        573511          77.0          1         25000.0           5   
        576643          71.0          1         25000.0           5   
        577159          67.0          1         15000.0           5   
        579557          62.0          1       2599000.0           5   
        579888          56.0          1      